# Homework 2 Beer Recommendations


In [ ]:
#Import everything.
import pandas as pd
import nltk
import matplotlib.pyplot as plt
from sklearn import manifold
from matplotlib.font_manager import FontProperties
import re

import nltk
from nltk import FreqDist
from nltk.corpus import stopwords # Import the stop word list
from nltk.tokenize import sent_tokenize, word_tokenize

from collections import Counter
from itertools import chain

In [ ]:
!pip install selenium
!apt-get -q update # to update ubuntu to correctly run apt install
!apt install -yq chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Reading package lists...
Reading package lists...
Building dependency tree...
Reading state information...
chromium-chromedriver 

In [ ]:
#Importing packages
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
import pandas as pd

In [ ]:
#load in driver
driver = webdriver.Chrome('chromedriver', options=chrome_options)

In [ ]:
driver.get('https://www.beeradvocate.com/beer/top-rated/')

In [ ]:
%%time

# Get the beers for the scraper
beer_list = []

x = 2
while x < 252:
    beer = driver.find_elements_by_xpath("//*[@id='ba-content']/table/tbody/tr[{}]/td[2]/a/b".format(x))[0].text
    beer_list.append(beer)
    x += 1

CPU times: user 517 ms, sys: 19.1 ms, total: 536 ms
Wall time: 7.89 s


In [ ]:
%%time
all_comments = pd.DataFrame(columns = ['Product_name','product_review', 'user_ratings', "overall_rating"]) 

page = 0

for beer in beer_list:
    print(page)
    
    driver.find_element_by_link_text("{}".format(beer)).click()
    driver.implicitly_wait(1)
     
    ids = driver.find_elements_by_xpath("//*[@id='rating_fullview_user']/div/a")
    
    rating = 0
    
    # Get rid of beers that have less than 25 reviews
    while rating <= (len(ids) - 1):

        product = beer
        
        # Gets the whole review with ratings and date since the xpath has those built in to the attribute
        # Can not just get the text of the review.
        review = driver.find_elements_by_xpath("//*[@id='rating_fullview_content_2']")[rating].text
        
        # gets user ratings element
        user_ratings = driver.find_elements_by_xpath("//*[@id='rating_fullview_content_2']/span[4]")[rating].text
        
        # Gets overall rating
        overall_rating = driver.find_elements_by_xpath("//*[@id='rating_fullview_content_2']/span[2]")[rating].text

        all_comments.loc[len(all_comments)] = [product,review,user_ratings,overall_rating]

        rating += 1
    
    page += 1
    driver.execute_script("window.history.go(-1)")
    driver.implicitly_wait(3)

In [ ]:
def clean_review(s):
    string = s.splitlines()
    string = list(filter(None, string))
    del string[0:2]
    del string[-2:]
    return string

In [ ]:
all_comments["clean_review"] = all_comments["product_review"].map(clean_review)

In [ ]:
all_comments.shape

In [ ]:
all_comments.to_csv("6205 beer reviews of 250 beers.csv", sep="\t")

### Task B

In [ ]:
beers = pd.read_csv("6205 beer reviews of 250 beers.csv", sep="\t")

In [ ]:
beers.head()

,Unnamed: 0,Product_name,product_review,user_ratings,overall_rating,clean_review
0,0,Kentucky Brunch Brand Stout,HOPTOMIC_BOMB from California\n\n5/5 rDev +2%...,+2%,5.00,['look: 5 | smell: 5 | taste: 5 | feel: 5 | ov...
1,1,Kentucky Brunch Brand Stout,Prospero from Colorado\n\n4.53/5 rDev -7.6%\n...,-7.6%,4.53,['look: 4.5 | smell: 4.5 | taste: 4.5 | feel: ...
2,2,Kentucky Brunch Brand Stout,BourbonForBeer from Illinois\n\n1.49/5 rDev -...,-69.6%,1.49,['look: 3 | smell: 1.5 | taste: 1.5 | feel: 1....
3,3,Kentucky Brunch Brand Stout,JellybonesElmer from Wisconsin\n\n4.52/5 rDev...,-7.8%,4.52,['look: 4 | smell: 4.5 | taste: 4.5 | feel: 5 ...
4,4,Kentucky Brunch Brand Stout,WormGod from New York\n\n4.99/5 rDev +1.8%\nl...,+1.8%,4.99,['look: 4.75 | smell: 5 | taste: 5 | feel: 5 |...


In [ ]:
beers.loc[1000,'clean_review']

"['look: 4.75 | smell: 4.75 | taste: 4.75 | feel: 4.75 | overall: 4.75', 'Poured into a 20 oz giant tulip glass bottled 3/13/2020. Pours midnight black in the dead of a cloudy Winters evening, with a massive 2 plus finger mocha head that leaves a few thin streaks of lace, as it slowly settles into a thin layer of mocha bubbles. 4.75.', 'Smell is deep dark fudge, vanilla beans, roasty coffee grounds, cinnamon, and some oaked bourbon. 4.75', 'Taste follows dark chocolate, vanilla, coffee, cinnamon, and a little bourbon lingers sip after glorious sip. 4.75', 'Mouthfeel is BIG, a tad dry, gentle carbonation, and at 10.2% it drinks bigger, but extremely approachable and complex. 4.75']"

It seems that the review also got a generic look, smell, state, feel, overall at the beginning of every review. We will most likely have to remove this later in our word frequency analysis

## Task B - Word Frequency Analysis

In [ ]:
#create a new data frame for beers. Clean up the reviews even more for word tokenization.
beers2 = beers[['Product_name','clean_review']].copy()

In [ ]:
#Replace punctuation with blanks. 
beers2['new_cleaned_review'] = beers2['clean_review'].str.replace(r'[^\w\s]+', '')

In [ ]:
#Convert to lower case
beers2['new_cleaned_review'] = beers2['new_cleaned_review'].str.lower()

Then we use nltk to tokenize the reviews into words. 

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
stopwords_list = stopwords.words('english')

#Function to get the word tokens
def get_tokens(entry):
    #Use nltk to get word tokens.
    tokens = word_tokenize(entry)
    
    #Tokenize the word here. Returns the word if the word is a string. .isalpha() is a method that does this.
    token_words = [w for w in tokens if w.isalpha()]
    
    return token_words

#Function to remove the stopwords
def remove_stop_words(entry):
    
    no_stopwords = [w for w in entry if not w in stopwords_list]
    return (no_stopwords)


#Function to delist the entries and join the words together in a string.
def rejoin_words(entry):
    
    joined_words = ( " ".join(entry))
    return joined_words



In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
#Apply functions
#Word tokenization first
beers2['words'] = beers2['new_cleaned_review'].apply(get_tokens)

#Remove stopwords next
beers2['words'] = beers2['words'].apply(remove_stop_words)

#Rejoin list to a sentence.
beers2['processed_review'] = beers2['words'].apply(rejoin_words)

#Iterate through the sentences.

#Use the Counter function from the collections module to count words.
#Use chain from itertools module to iterate.
tally = Counter(chain.from_iterable(map(str.split, beers2['processed_review'].tolist()))) 

#Create a series for the word counts.
series = pd.Series(tally).sort_values(ascending=False)

#Create a term frequency dataframe for the words and their frequency
term_freq = series.reset_index()
term_freq.columns = ['words','freq']


term_freq.to_csv('beers_term_frequency.csv')

In [ ]:
#Top ten most mentioned words
term_freq.head(10)

,words,freq
0,taste,7894
1,smell,6986
2,feel,6852
3,look,6546
4,overall,6480
5,head,2586
6,beer,1972
7,dark,1856
8,chocolate,1792
9,sweet,1388


Due to how the reviews are structured, it seems like taste, smell, feel, look, and overall are mentioned in every review. This makes them not that useful for our analysis so we need to get rid of them.

In [ ]:
term_freq2 = term_freq.iloc[5:]

In [ ]:
term_freq2.head(10)

,words,freq
5,head,2586
6,beer,1972
7,dark,1856
8,chocolate,1792
9,sweet,1388
10,light,1304
11,notes,1303
12,coffee,1279
13,vanilla,1279
14,like,1265


From our word frequency analysis, we can see the most commonly reviewed words that are similar to attributes for beer referenced by dummies.com. We elected to choose two of those attributes: balanced and fruity for our attribute list as well as light for our three beer attributes in our reccomendation algorithim.

##Task C 
Cosine Similarity Analysis
Balanced, Fruity, Light


In [ ]:
!python -m spacy download en_core_web_lg

     |████████████████████████████████| 827.9MB 1.2MB/s 
  Created wheel for en-core-web-lg: filename=en_core_web_lg-2.2.5-cp36-none-any.whl size=829180944 sha256=c45037f4084fde8f7da2a97ecccf0bbe03f2bb8a0dacf4ba21ddd1f5476fff8b
  Stored in directory: /tmp/pip-ephem-wheel-cache-gtkr9s9i/wheels/2a/c1/a6/fc7a877b1efca9bc6a089d6f506f16d3868408f9ff89f8dbfc
Successfully built en-core-web-lg
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [ ]:
attributes = ['balanced','fruity','light']

In [ ]:
import spacy
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import en_core_web_lg

In [ ]:
nlp = spacy.load('en_core_web_lg')

In [ ]:
attributes_nlp = nlp('balanced, fruity, light')
def get_similarity(review_words):
    text_review = ' '.join(review_words)
    review_nlp = nlp(text_review)
    return review_nlp.similarity(attributes_nlp)

beers2['similarity'] =  beers2['new_cleaned_review'].apply(get_similarity)

/usr/lib/python3.6/runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
/usr/lib/python3.6/runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
/usr/lib/python3.6/runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
/usr/lib/python3.6/runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
/usr/lib/python3.6/runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)


In [ ]:
beers2.groupby('Product_name')['similarity'].mean().reset_index().sort_values(by='similarity',ascending = False).head(5)

,Product_name,similarity
105,Fort Point Pale Ale,0.269582
85,Double Dry Hopped Mylar Bags,0.268095
205,Scaled Up,0.267754
81,Double Dry Hopped Double Mosaic Daydream,0.267630
183,Oude Geuze Cuvée Armand & Gaston,0.267412


## Task D

Sentiment analysis based on attributes balanced, fruity, light.

In [ ]:
pip install nltk


In [ ]:
pip install VADER

     |████████████████████████████████| 51kB 1.7MB/s 
  Created wheel for sonopy: filename=sonopy-0.1.2-cp36-none-any.whl size=2881 sha256=33affe5e1e4651ffaf9da9bcb261f7004cedf432144c079e6c3fb4e961f9e618
  Stored in directory: /root/.cache/pip/wheels/b6/39/ba/b2f21d4fbcb362658c73f83c9502782300b0399aef3693b506
Successfully built sonopy


In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

/usr/local/lib/python3.6/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [ ]:
from nltk.tokenize import sent_tokenize
import nltk

In [ ]:
#Create dataframe with original reviews
sent_beers = beers[['Product_name','clean_review']].copy()
#Add in columns to store sentiment for the attributes
sent_beers['balanced_sentiment'] = 0
sent_beers['fruity_sentiment'] = 0
sent_beers['light_sentiment'] = 0

In [ ]:
nltk.download('vader_lexicon')
vader = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


In [ ]:
#Iterrate through the dataframe
for i in range(len(sent_beers)):
    #Get a review and split it into sentence tokens
    review = sent_beers.loc[i,'clean_review']
    review = sent_tokenize(review)
    
    #Store score values here
    #Resets every time loop runs. This is intentional.
    comp_balanced = []
    comp_fruity = []
    comp_light = []
    
    #Iterate through the sentence tokens. Get compound sentiment score if the word is present.
    for sent_token in review:
        if 'balanced' in sent_token.lower():
            bal_score = vader.polarity_scores(sent_token).get('compound')
            comp_balanced.append(bal_score)
            
        if 'fruity' in sent_token.lower():
            fruity_score = vader.polarity_scores(sent_token).get('compound')
            comp_fruity.append(fruity_score)
            
        if 'light' in sent_token.lower():
            light_score = vader.polarity_scores(sent_token).get('compound')
            comp_light.append(light_score)
    
    #Store each sentiment score in the array
    if len(comp_balanced) == 0: #Nothing is in the list. No sentiment for the review.
        sent_beers.loc[i,'balanced_sentiment'] = None
    else: #Balanced is in the review. Take the average of the list and store it.
        #Taking average means to convert it into a numpy array and then using the np.mean
        sent_beers.loc[i,'balanced_sentiment'] = np.mean(np.array(comp_balanced))
    
    #Repeat conditions for fruity and light.
    if len(comp_fruity) == 0:
        sent_beers.loc[i,'fruity_sentiment'] = None
    else:
        sent_beers.loc[i,'fruity_sentiment'] = np.mean(np.array(comp_fruity))
    if len(comp_light) == 0:
        sent_beers.loc[i,'light_sentiment'] = None
    else:
        sent_beers.loc[i,'light_sentiment'] = np.mean(np.array(comp_light))

In [ ]:
#Group all the sentiments of the beer together
avg_sent_beers = sent_beers.groupby(by=["Product_name"])[["balanced_sentiment", "fruity_sentiment", "light_sentiment"]].mean()

In [ ]:
#Sentiments that are not present are treated as Zero. 
#Want to pinpoint beers that are balanced, fruity, and light.
#Mean is simply adding three columns together and dividng by 3
avg_sent_beers['average_sentiment'] = (avg_sent_beers['balanced_sentiment'].fillna(0)+avg_sent_beers['fruity_sentiment'].fillna(0)
 +avg_sent_beers['light_sentiment'].fillna(0))/3

In [ ]:
avg_sent_beers.sort_values(by='average_sentiment', ascending = False)

,balanced_sentiment,fruity_sentiment,light_sentiment,average_sentiment
Product_name,,,,
Beyond Good And Evil,0.70015,0.968900,0.601817,0.756956
A Deal With The Devil,0.78720,0.992300,0.380000,0.719833
Oude Geuze Golden Blend,0.71015,0.881125,0.277375,0.622883
Framboise Du Fermier,0.86250,NaN,0.862500,0.575000
Abt 12,0.74980,0.443775,0.494524,0.562700
...,...,...,...,...
Grande Negro Voodoo Papi - Bourbon Barrel-Aged,-0.24110,-0.165500,0.222375,-0.061408
Birth Of Tragedy,-0.61130,0.295150,0.010963,-0.101729
Scaled Up,-0.40190,NaN,0.052567,-0.116444


## Task E

In [ ]:
eval_df = beers2.groupby('Product_name')['similarity'].mean()

In [ ]:
eval_df = eval_df.reset_index()

In [ ]:
eval_df = eval_df.merge(avg_sent_beers, left_on = 'Product_name',right_on='Product_name')

In [ ]:
eval_df['eval_score'] = eval_df['average_sentiment'] + eval_df['similarity']

In [ ]:
eval_df.sort_values(by ='eval_score', ascending = False).head(3)

,Product_name,similarity,balanced_sentiment,fruity_sentiment,light_sentiment,average_sentiment,eval_score
30,Beyond Good And Evil,0.262348,0.70015,0.968900,0.601817,0.756956,1.019304
2,A Deal With The Devil,0.264347,0.78720,0.992300,0.380000,0.719833,0.984181
184,Oude Geuze Golden Blend,0.265702,0.71015,0.881125,0.277375,0.622883,0.888586


From our eval_score we added up the cosine similarity and average sentiment. We would reccomend the above 3 products since they have highest eval scores.

## Task F

Use word vectors instead of cosine similarity.

In [ ]:
!python -m spacy download en_core_web_md

     |████████████████████████████████| 96.4MB 1.2MB/s 
  Created wheel for en-core-web-md: filename=en_core_web_md-2.2.5-cp36-none-any.whl size=98051305 sha256=0fea264060019d61a87c580a9561a88b32b6da503f5b9e7d5d9c666b0a84cd2e
  Stored in directory: /tmp/pip-ephem-wheel-cache-lskvxvzq/wheels/df/94/ad/f5cf59224cea6b5686ac4fd1ad19c8a07bc026e13c36502d81
Successfully built en-core-web-md
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_md')


In [ ]:
from nltk.tokenize import PunktSentenceTokenizer, RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from tempfile import NamedTemporaryFile
import shutil
import warnings
import spacy
import en_core_web_md
import csv
import pandas as pd
import nltk
import numpy as np
from nltk.tokenize import PunktSentenceTokenizer, RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from tempfile import NamedTemporaryFile
import shutil
import warnings
import spacy
import en_core_web_md

In [ ]:
nlp = en_core_web_md.load()

In [ ]:
attributes

['balanced', 'fruity', 'light']

In [ ]:
from pandas import DataFrame
from nltk.tokenize import word_tokenize
import string

In [ ]:
beers2['processed_review'].str.cat(sep=' ')

'look smell taste feel overall smell early morning pancakes coffee work wakes taste liquifies buttery maple pancakes hot coffee snake bite feel like scene blood oil explodes everywhere tar look smell taste feel overall look smell taste feel overall hyped lot breweries style beer right top rated kinda nuts rated hyped priced found lot local beers much less tbh gave overall worth msrp whatsoever beer friends brewed balanced look smell taste feel overall look smell taste feel overall look smell taste feel overall expensive draft pour tg release day dec look dark expected thin brownishtan head sticks glass briefly smell lots phenomenal coffee hit first much stronger mornin delight barrel along dark chocolate taste coffee maple match perfectly vanilla warmed barrel present really delicious feel carbonation great unbelievably smooth lingered sip overall easily best beer ive tg complex drinkable everything wanted beer many stouts maple none mix flavors like definitely world class beer look sm

In [ ]:
beers2['processed_review'] = beers2['processed_review'].astype(str)

In [ ]:
warnings.filterwarnings("ignore", category = DeprecationWarning)

reviews_df = beers2[['processed_review','Product_name']].copy()

reviews_df['vector_similarity'] = ''

for index, row in reviews_df.iterrows():
  doc1 = nlp(' '.join(attributes))
  doc2 = nlp(row['processed_review'])
  reviews_df.loc[index,'vector_similarity'] = doc1.similarity(doc2)


/usr/lib/python3.6/runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
/usr/lib/python3.6/runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
/usr/lib/python3.6/runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
/usr/lib/python3.6/runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
/usr/lib/python3.6/runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)


In [ ]:
reviews_df.sort_values(by = 'vector_similarity', ascending = False).head(5)

,processed_review,Product_name,vector_similarity
5812,look smell taste feel overall release pours pu...,Coolship Red,0.853817
5846,look smell taste feel overall light gold color...,Cellarman Barrel Aged Saison,0.850791
3603,look smell taste feel overall look classic neh...,Heavy Mettle,0.845505
704,look smell taste feel overall bottle poured te...,Duck Duck Gooze,0.833874
5556,look smell taste feel overall excellent cloudy...,Vicinity,0.832596


In [ ]:
reviews_df['vector_similarity'] = reviews_df['vector_similarity'].astype(float)

In [ ]:
vector_sim_df = reviews_df.groupby('Product_name')['vector_similarity'].mean()
comparison_df = vector_sim_df.reset_index()
comparison_df = comparison_df.merge(eval_df, left_on = 'Product_name',right_on='Product_name')

In [ ]:
comparison_df['cosine_similarity'] = comparison_df['similarity']

In [ ]:
comparison_df = comparison_df.drop(columns = ['similarity'])

In [ ]:
comparison_df['vector_eval'] = comparison_df['vector_similarity']+comparison_df['average_sentiment']

In [ ]:
comparison_df[['Product_name','vector_similarity','cosine_similarity','average_sentiment','eval_score','vector_eval']].sort_values(by = 'vector_eval', ascending = False).head(5)

,Product_name,vector_similarity,cosine_similarity,average_sentiment,eval_score,vector_eval
30,Beyond Good And Evil,0.688228,0.262348,0.756956,1.019304,1.445183
2,A Deal With The Devil,0.694848,0.264347,0.719833,0.984181,1.414682
184,Oude Geuze Golden Blend,0.720517,0.265702,0.622883,0.888586,1.343401
9,Abt 12,0.717062,0.263197,0.562700,0.825896,1.279762
130,Hefeweissbier,0.708803,0.261631,0.551494,0.813125,1.260297


In [ ]:
contain_df = beers2[['Product_name','processed_review']].copy()

In [ ]:
def contains_script(attribute1, attribute2, attribute3,df):
  df['contains ' + attribute1] = 0
  df['contains ' + attribute2] = 0
  df['contains ' + attribute3] = 0
  reviews = list(df['processed_review'].values)

  for review in reviews:
    if attribute1 in review:
      df.loc[reviews.index(review), 'contains ' + attribute1] = 1
    else:
      df.loc[reviews.index(review), 'contains ' + attribute1] = 0
    if attribute2 in review:
      df.loc[reviews.index(review), 'contains ' + attribute2] = 1
    else:
      df.loc[reviews.index(review), 'contains ' + attribute2] = 0
    if attribute3 in review:
      df.loc[reviews.index(review), 'contains ' + attribute3] = 1
    else:
      df.loc[reviews.index(review), 'contains ' + attribute3] = 0
  return df

In [ ]:
contain_df = contains_script('balanced', 'fruity', 'light',contain_df)

In [ ]:
def D(row):
  if row['contains balanced'] + row['contains fruity'] + row['contains light'] == 0:
    val = 0
  else:
    val = 1
  return val

contain_df['contains_attribute'] = contain_df.apply(D, axis = 1)

In [ ]:
mean_contain = contain_df.groupby('Product_name').mean()

In [ ]:
comparison_df = comparison_df.merge(mean_contain, left_on = 'Product_name',right_on='Product_name')

In [ ]:
comparison_df[
['Product_name','contains balanced','contains fruity','contains light','contains_attribute','eval_score','vector_eval']
].sort_values(by = 'eval_score', ascending = False).head(5)

,Product_name,contains balanced,contains fruity,contains light,contains_attribute,eval_score,vector_eval
30,Beyond Good And Evil,0.04,0.04,0.20,0.20,1.019304,1.445183
2,A Deal With The Devil,0.08,0.04,0.28,0.32,0.984181,1.414682
184,Oude Geuze Golden Blend,0.08,0.08,0.28,0.32,0.888586,1.343401
108,Framboise Du Fermier,0.04,0.00,0.04,0.04,0.832776,1.233647
9,Abt 12,0.04,0.16,0.32,0.40,0.825896,1.279762


In [ ]:
comparison_df[
['Product_name','contains balanced','contains fruity','contains light','contains_attribute','eval_score','vector_eval']
].sort_values(by = 'vector_eval', ascending = False).head(5)

,Product_name,contains balanced,contains fruity,contains light,contains_attribute,eval_score,vector_eval
30,Beyond Good And Evil,0.04,0.04,0.20,0.20,1.019304,1.445183
2,A Deal With The Devil,0.08,0.04,0.28,0.32,0.984181,1.414682
184,Oude Geuze Golden Blend,0.08,0.08,0.28,0.32,0.888586,1.343401
9,Abt 12,0.04,0.16,0.32,0.40,0.825896,1.279762
130,Hefeweissbier,0.04,0.04,0.24,0.24,0.813125,1.260297


Based on our attributes of Balanced, Fruity, and Light, the top 3 recommended beers are listed above. In both cosine similarity and in word vecotrs, we found that the top 3 beers are still the same and thus we would reccomend the same product. However, if we were going by the top 5 beers, then the reccomendations would be different between the two. We also took a look at the percent of reviews given a product that mentioned a preferred attribute. As it turns out, the top 3 beers don't really have balanced or fruity. This is most likely because the reviews themselves don't use those words as seen in our word frequency analysis. However, we do get a good occurrence of light which was an attribute that we picked from our top word frequency analysis. To improve the other attributes, we think that changing fruity to citrus would be better overall since that word is used more to describe fruity beer.

## Task G - Recommending changes


Compare overall ratings to text analysis.

In [ ]:
beer_original = pd.read_csv("6205 beer reviews of 250 beers.csv", sep="\t")
beers_partg= beer_original[['Product_name', 'overall_rating']].copy()
beers_partg['user_ratings'] = beers_partg['overall_rating']

In [ ]:
user_beers = beers_partg.groupby('Product_name')['user_ratings'].mean().reset_index()

In [ ]:
user_beers.head(5)

,Product_name,user_ratings
0,3rd Anniversary Imperial IPA,4.578800
1,4th Anniversary,4.615600
2,A Deal With The Devil,4.581600
3,A Deal With The Devil - Double Oak-Aged,4.722727
4,Aaron,4.565600


In [ ]:
comparison_df = comparison_df.merge(user_beers, left_on = 'Product_name',right_on='Product_name')

In [ ]:
comparison_df[
['Product_name','eval_score','vector_eval','user_ratings']
].sort_values(by = 'user_ratings', ascending = False).head(5)

,Product_name,eval_score,vector_eval,user_ratings
56,Chemtrailmix,0.530059,0.961241,4.754286
142,It Was All A Dream,0.450179,0.870967,4.747200
199,SR-71,0.680172,1.098754,4.746400
26,Barrel-Aged Abraxas,0.253105,0.671092,4.744400
14,Ann,0.745104,1.192627,4.725200


In [ ]:
comparison_df[
['Product_name','eval_score','vector_eval','user_ratings']
].sort_values(by = 'eval_score', ascending = False).head(5)

,Product_name,eval_score,vector_eval,user_ratings
30,Beyond Good And Evil,1.019304,1.445183,4.4960
2,A Deal With The Devil,0.984181,1.414682,4.5816
184,Oude Geuze Golden Blend,0.888586,1.343401,4.5632
108,Framboise Du Fermier,0.832776,1.233647,4.5032
9,Abt 12,0.825896,1.279762,4.5064


The recommendations based on the user ratings do not match at all with our results from using cosine similarity and spacy word2vec analysis. These products would not match what the user wants as their evaluation scores are rather low. This means that the beers are not that associated with the three attributes that the user wants. This is not that surprising since user ratings are very broad and not pinpointing specific attributes.
